In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import os
import re
import glob
import cv2
import random
import time
import gc

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# torchvision
import torchvision
from torchvision import transforms, datasets

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

import timm
from timm import create_model

import torchmetrics

from sklearn.preprocessing import LabelEncoder

# Utils
from tqdm.auto import tqdm, trange

import warnings
warnings.filterwarnings(action='ignore')

In [11]:
# timm에서 사용할 수 있는 모델 리스트 불러오기
size = (3,3,3)
torch.randn(size)

tensor([[[-0.5581,  1.3625, -0.4348],
         [-0.4470,  1.5754,  0.9783],
         [ 0.6502, -1.2192,  1.4294]],

        [[ 0.7953,  0.4238, -1.9113],
         [ 0.3517,  1.6569,  0.9577],
         [ 0.7508,  2.1028,  0.4501]],

        [[-1.3490,  0.2560, -0.2655],
         [ 0.3859,  0.7323,  0.0037],
         [ 0.1820, -1.5374,  0.3087]]])

In [12]:
torch.randn(*size)

tensor([[[ 0.9448, -0.9426,  1.3304],
         [ 0.7952,  0.1989,  0.1005],
         [ 0.1136, -1.4487, -1.2026]],

        [[ 0.1708,  0.6990,  0.1465],
         [ 0.9509, -0.8060,  0.4238],
         [-1.4892,  1.8895,  0.7065]],

        [[ 1.0684,  0.8743, -0.3099],
         [-0.9624, -2.4079, -0.5602],
         [-0.2960,  1.4314, -1.8829]]])

## Config
 - 'Config' dictionary를 셍성합니다.
 - Python Script로 작성할 때 유용합니다.
 - Train 혹은 Inference 할 때 필요한 파라미터값들을 지정  
   : Epochs, Batch Size, Learning Rate, ...

In [9]:
config ={
    'model': ,
    'model_save' : '/content/',
    'data_path' : ,
    'learning_rate': ,
    'seed': 2023,
    'img_size': ,
    'n_epochs': ,
    'ratio': ,
    "train_batch_size": ,
    "valid_batch_size": ,
    "min_lr": ,
    "T_max": ,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
}


SyntaxError: expression expected after dictionary key and ':' (3781809392.py, line 2)

## Data

In [32]:
all_img_list = glob.glob("./data/" +'train/*/*')
# all_img_list = glob.glob(config['data_path'] +'train/*/*')
print(len(all_img_list))
all_img_list[:3]

3457


['./data/train\\가구수정\\0.png',
 './data/train\\가구수정\\1.png',
 './data/train\\가구수정\\10.png']

In [33]:
sample = all_img_list[33]
sample

'./data/train\\걸레받이수정\\117.png'

In [51]:
train = pd.DataFrame(columns=['img_path', 'label'])
train['img_path'] = all_img_list
train['img_path'].replace("\\", "/")
train['label'] = train['img_path'].apply(lambda x : str(x).split('/')[-2])
print(train.shape)
train.head()

(3457, 2)


,img_path,label
0,./data/train\가구수정\0.png,data
1,./data/train\가구수정\1.png,data
2,./data/train\가구수정\10.png,data
3,./data/train\가구수정\11.png,data
4,./data/train\가구수정\2.png,data


## Sample

'./data/train/걸레받이수정/117.png'

In [ ]:
num = 2223
sample = train.img_path[num]
image = cv2.imread(sample)
print("Sample Image")
plt.imshow(image)

## Target Encoding
 - Classification 문제이기 때문에, Target의 Class를 파악합니다.
 - Class를 0, 1, 2, 3, ... 으로 인코딩을 합니다.

In [20]:
le = LabelEncoder()

In [21]:
train['new_target'] = le.fit_transform(train['label'])

In [22]:
train.head()

,img_path,label,new_target
0,./data/train\가구수정\0.png,data,0
1,./data/train\가구수정\1.png,data,0
2,./data/train\가구수정\10.png,data,0
3,./data/train\가구수정\11.png,data,0
4,./data/train\가구수정\2.png,data,0


In [23]:
train.label.value_counts()

label
data    3457
Name: count, dtype: int64

In [24]:
train.new_target.value_counts()

new_target
0    3457
Name: count, dtype: int64

In [25]:
n_classes = train.new_target.nunique()
n_classes

1

## Set Seed

 > 실험적인 시도 혹은 Model Ensemble할 때 유용합니다.

In [26]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
## Set Seed
set_seed(config['seed'])

## Augmentation